# Format the ARCS data for PointPillars

In [35]:
import os
import random

# Make a test/train/val split lists, save them in image sets

In [37]:
ARCS_LABEL_DIR = '..\\data\\corrected_labels'
ARCS_VELODYNE_DIR = '..\\data\\velodyne_points'
ARCS_IMAGE_DIR = '..\\data\\image'
BASE_PATH = '..\\data'

In [38]:
def create_directory_tree():
    # Define the structure of the directory tree
    structure = {
        'split_arcs': {
            'ImageSets': {},
            'testing': {
                'calib': {},
                'image_2': {},
                'label_2': {},
                'velodyne': {}
            },
            'training': {
                'calib': {},
                'image_2': {},
                'label_2': {},
                'velodyne': {}
            }
        }
    }
    
    def create_subdirectories(parent_path, subdirs):
        for subdir, children in subdirs.items():
            path = os.path.join(parent_path, subdir)
            os.makedirs(path, exist_ok=True)  # Create the directory if it doesn't exist
            create_subdirectories(path, children)  # Recursively create child directories

    # Start creating directories from the base path
    create_subdirectories(BASE_PATH, structure)

In [39]:
# First get a list of file names from labels
def get_labels(labels_path):
    # List all files in the directory
    file_list = [file for file in os.listdir(labels_path) if
                 os.path.isfile(os.path.join(labels_path, file))]
    return file_list

In [40]:
# Function to save lists to files
def save_list_to_file(list_of_files, file_name):
    with open(file_name, 'w') as f:
        total_items = len(list_of_files)
        for index, item in enumerate(list_of_files):
            # Split the file name at the dot and take the first part
            numeric_part = item.split('.')[0]
            if index == total_items - 1:
                f.write(numeric_part)  # Write without newline
            else:
                f.write("%s\n" % numeric_part)  # Write with newline

In [41]:
# Make a train, val, test split, save lists to arcs/ImageSets
def save_train_val_test_splits(label_files_list):
    # Shuffle the list to ensure randomness
    random.shuffle(label_files_list)

    # Define split proportions
    train_split = 0.65  # 65% of the data
    val_split = 0.2  # 20% of the data
    test_split = 0.15  # 15% of the data

    # Calculate split sizes
    total_files = len(label_files_list)
    train_size = int(total_files * train_split)
    val_size = int(total_files * val_split)

    # Split the dataset
    train_files = label_files_list[:train_size]
    val_files = label_files_list[train_size:train_size + val_size]
    test_files = label_files_list[train_size + val_size:]

    # Combine train and val for trainval
    trainval_files = train_files + val_files

    # Make
    root = os.path.join(BASE_PATH, 'split_arcs', 'ImageSets')
    os.makedirs(root, exist_ok=True)

    # Save to files
    save_list_to_file(train_files, os.path.join(root, 'train.txt'))
    save_list_to_file(val_files, os.path.join(root, 'val.txt'))
    save_list_to_file(test_files, os.path.join(root, 'test.txt'))
    save_list_to_file(trainval_files, os.path.join(root, 'trainval.txt'))

In [42]:
create_directory_tree()
label_files_list = get_labels(ARCS_LABEL_DIR)
save_train_val_test_splits(label_files_list)

# Using those lists, move/copy the .bin and .txt files to the appropriate folders

In [44]:
import shutil

In [45]:
def copy_files(label_list_file, list_file_path, label_dest_path, bin_dest_path, image_path_dest, calib_path_dest):
    # Image default file source
    image_default_src = os.path.join('default_files', 'placeholder.png')
    calib_default_src = os.path.join('default_files', 'placeholder_calib.txt')

    # Open the file containing the list of files to copy
    with open(list_file_path, 'r') as file_list:
        for file_name in file_list:
            file_name = file_name.strip()  # Remove any leading/trailing whitespace

            # Copy labels
            txt_file_name = file_name + '.txt'
            txt_src_file_path = os.path.join(ARCS_LABEL_DIR, txt_file_name)
            txt_dest_file_path = os.path.join(label_dest_path, txt_file_name)
            shutil.copy2(txt_src_file_path, txt_dest_file_path)

            # Copy velodyne points
            bin_file_name = file_name + '.bin'
            bin_src_file_path = os.path.join(ARCS_VELODYNE_DIR, bin_file_name)
            bin_dest_file_path = os.path.join(bin_dest_path, bin_file_name)
            shutil.copy2(bin_src_file_path, bin_dest_file_path)
            
            # # Then copy a placeholder image
            # png_file_name = file_name + '.png'
            # png_dest_file_path = os.path.join(image_path_dest, png_file_name)
            # shutil.copy(image_default_src, png_dest_file_path)
            
            # Copy over images
            png_file_name = file_name + '.png'
            png_src_file_path = os.path.join(ARCS_IMAGE_DIR, png_file_name)
            png_dest_file_path = os.path.join(image_path_dest, png_file_name)
            shutil.copy(png_src_file_path, png_dest_file_path)

            # Copy the calib files
            calib_dest_file_path = os.path.join(calib_path_dest, txt_file_name)
            shutil.copy(calib_default_src, calib_dest_file_path)

In [46]:
def copy_set_up(label_list_file, target_dir):
    # Define the source and destination directories for trainval or test files
    index_src_path = os.path.join(BASE_PATH, 'split_arcs', 'ImageSets', label_list_file)
    label_dest_path = os.path.join(BASE_PATH, 'split_arcs', target_dir, 'label_2')
    bin_dest_path = os.path.join(BASE_PATH, 'split_arcs', target_dir, 'velodyne')

    image_dest_path = os.path.join(BASE_PATH, 'split_arcs', target_dir, 'image_2')
    calib_dest_path = os.path.join(BASE_PATH, 'split_arcs', target_dir, 'calib')
    # Make sure directory exists
    os.makedirs(label_dest_path, exist_ok=True)
    os.makedirs(bin_dest_path, exist_ok=True)
    os.makedirs(image_dest_path, exist_ok=True)
    os.makedirs(calib_dest_path, exist_ok=True)

    # Copy files
    copy_files(label_list_file, index_src_path, label_dest_path, bin_dest_path, image_dest_path, calib_dest_path)

In [47]:
def copy_files_to_split_directory():
    copy_set_up('test.txt', 'testing')
    copy_set_up('trainval.txt', 'training')

In [48]:
copy_files_to_split_directory()

# Reindex the lists? (May not be necessary)

# Zip